In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

patients = pd.read_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PATIENTS.csv.gz")
admissions = pd.read_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz")
icustays = pd.read_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ICUSTAYS.csv.gz")
labevents = pd.read_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/LABEVENTS.csv.gz")

patients.head(), admissions.head(), icustays.head(), labevents.head()


(   ROW_ID  SUBJECT_ID GENDER                  DOB                  DOD  \
 0     234         249      F  2075-03-13 00:00:00                  NaN   
 1     235         250      F  2164-12-27 00:00:00  2188-11-22 00:00:00   
 2     236         251      M  2090-03-15 00:00:00                  NaN   
 3     237         252      M  2078-03-06 00:00:00                  NaN   
 4     238         253      F  2089-11-26 00:00:00                  NaN   
 
               DOD_HOSP DOD_SSN  EXPIRE_FLAG  
 0                  NaN     NaN            0  
 1  2188-11-22 00:00:00     NaN            1  
 2                  NaN     NaN            0  
 3                  NaN     NaN            0  
 4                  NaN     NaN            0  ,
    ROW_ID  SUBJECT_ID  HADM_ID            ADMITTIME            DISCHTIME  \
 0      21          22   165315  2196-04-09 12:26:00  2196-04-10 15:54:00   
 1      22          23   152223  2153-09-03 07:15:00  2153-09-08 19:10:00   
 2      23          23   124321  2

In [3]:
import pandas as pd
import numpy as np

# =====================
# 0. Load data
# =====================
patients = pd.read_csv(
    "/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PATIENTS.csv.gz",
    usecols=["SUBJECT_ID", "GENDER", "DOB"],
    parse_dates=["DOB"],
)

admissions = pd.read_csv(
    "/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz",
    usecols=["HADM_ID", "SUBJECT_ID", "ADMITTIME", "DEATHTIME", "HOSPITAL_EXPIRE_FLAG"],
    parse_dates=["ADMITTIME", "DEATHTIME"],
)

icustays = pd.read_csv(
    "/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ICUSTAYS.csv.gz",
    usecols=["ICUSTAY_ID", "SUBJECT_ID", "HADM_ID", "INTIME", "OUTTIME"],
    parse_dates=["INTIME", "OUTTIME"],
)

# =====================
# 1. First ICU stay per subject
# =====================
icustays = icustays.sort_values(["SUBJECT_ID", "INTIME"])
first_icu = icustays.groupby("SUBJECT_ID", as_index=False).first()

# =====================
# 2. Merge tables
# =====================
cohort = (
    first_icu
    .merge(patients, on="SUBJECT_ID", how="left")
    .merge(admissions, on=["SUBJECT_ID", "HADM_ID"], how="left")
)

# =====================
# 3. Age calculation (DAY precision, overflow-safe)
# =====================
# 強制轉 datetime（錯的會變 NaT）
cohort["INTIME"] = pd.to_datetime(cohort["INTIME"], errors="coerce")
cohort["DOB"] = pd.to_datetime(cohort["DOB"], errors="coerce")

age = np.full(len(cohort), np.nan, dtype="float64")

m = cohort["INTIME"].notna() & cohort["DOB"].notna()

intime_d = cohort.loc[m, "INTIME"].values.astype("datetime64[D]")
dob_d    = cohort.loc[m, "DOB"].values.astype("datetime64[D]")

age_days = (intime_d - dob_d).astype("timedelta64[D]").astype("int64")
age[m.to_numpy()] = age_days / 365.25

cohort["age"] = age

# MIMIC 去識別處理：>89 歲 cap 成 90（論文常見做法）
cohort.loc[cohort["age"] > 89, "age"] = 90

# 只留成人
cohort = cohort[(cohort["age"] >= 18) & cohort["age"].notna()].copy()

# =====================
# 4. 30-day mortality label
# =====================
cohort["DEATHTIME"] = pd.to_datetime(cohort["DEATHTIME"], errors="coerce")

cohort["death_30d"] = 0
m2 = cohort["DEATHTIME"].notna() & cohort["INTIME"].notna()

cohort.loc[m2, "death_30d"] = (
    (cohort.loc[m2, "DEATHTIME"] - cohort.loc[m2, "INTIME"])
    .dt.total_seconds()
    <= 30 * 24 * 3600
).astype(int)

# =====================
# 5. Final modeling table
# =====================
cohort = cohort[
    [
        "SUBJECT_ID",
        "HADM_ID",
        "ICUSTAY_ID",
        "INTIME",
        "age",
        "GENDER",
        "death_30d",
    ]
].reset_index(drop=True)

# =====================
# 6. Sanity check
# =====================
print("Cohort shape:", cohort.shape)
print("30-day mortality rate:", cohort["death_30d"].mean())
print(cohort["age"].describe())
cohort.head()


Cohort shape: (38509, 7)
30-day mortality rate: 0.10896154145784102
count    38509.000000
mean        63.810317
std         17.472679
min         18.020534
25%         52.416153
50%         65.694730
75%         77.889117
max         90.000000
Name: age, dtype: float64


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,age,GENDER,death_30d
0,3,145834,211552,2101-10-20 19:10:11,76.522930,M,0
1,4,185777,294638,2191-03-16 00:29:31,47.843943,F,0
2,6,107064,228232,2175-05-30 21:30:54,65.938398,F,0
3,9,150750,220597,2149-11-09 13:07:02,41.787817,M,1
4,11,194540,229441,2178-04-16 06:19:32,50.146475,F,0


In [4]:
# =====================
# Day 2: 24h Lab Features
# =====================
labevents = pd.read_csv(
    "/content/drive/MyDrive/mimic-iii-clinical-database-1.4/LABEVENTS.csv.gz",
    usecols=["SUBJECT_ID", "HADM_ID", "ITEMID", "CHARTTIME", "VALUENUM"],
    parse_dates=["CHARTTIME"],
)

# ---- 常用 lab ITEMID（MIMIC-III）----
lab_itemids = {
    "creatinine": [50912],
    "bun": [51006],
    "wbc": [51301],
    "hemoglobin": [51222],
    "platelet": [51265],
    "sodium": [50983],
    "potassium": [50971],
    "glucose": [50931],
    "lactate": [50813],  # 如果太 sparse 可以之後丟掉
}

lab_df = labevents[
    labevents["ITEMID"].isin(sum(lab_itemids.values(), []))
].dropna(subset=["VALUENUM"])

# ---- join ICU intime ----
lab_df = lab_df.merge(
    cohort[["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID", "INTIME"]],
    on=["SUBJECT_ID", "HADM_ID"],
    how="inner",
)

# ---- 24hr window ----
lab_df["hours_from_icu"] = (
    (lab_df["CHARTTIME"] - lab_df["INTIME"]).dt.total_seconds() / 3600
)

lab_df = lab_df[(lab_df["hours_from_icu"] >= 0) & (lab_df["hours_from_icu"] <= 24)]

# ---- map ITEMID -> lab name ----
itemid_to_lab = {
    itemid: lab
    for lab, ids in lab_itemids.items()
    for itemid in ids
}

lab_df["lab"] = lab_df["ITEMID"].map(itemid_to_lab)

# ---- aggregate ----
lab_feat = (
    lab_df
    .groupby(["ICUSTAY_ID", "lab"])["VALUENUM"]
    .agg(["min", "max", "mean"])
    .reset_index()
)

lab_feat.columns = ["ICUSTAY_ID", "lab", "lab_min", "lab_max", "lab_mean"]

# ---- pivot wide ----
lab_feat_wide = lab_feat.pivot(
    index="ICUSTAY_ID",
    columns="lab",
    values=["lab_min", "lab_max", "lab_mean"]
)

lab_feat_wide.columns = [
    f"{stat}_{lab}" for stat, lab in lab_feat_wide.columns
]

lab_feat_wide = lab_feat_wide.reset_index()

# ---- merge back to cohort ----
model_df = cohort.merge(lab_feat_wide, on="ICUSTAY_ID", how="left")

print(model_df.shape)
model_df.head()


(38509, 34)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,age,GENDER,death_30d,lab_min_bun,lab_min_creatinine,lab_min_glucose,...,lab_max_wbc,lab_mean_bun,lab_mean_creatinine,lab_mean_glucose,lab_mean_hemoglobin,lab_mean_lactate,lab_mean_platelet,lab_mean_potassium,lab_mean_sodium,lab_mean_wbc
0,3,145834,211552,2101-10-20 19:10:11,76.522930,M,0,41.0,2.4,157.0,...,24.4,42.0,2.466667,223.25,9.025,4.871429,255.25,4.26,140.0,18.40
1,4,185777,294638,2191-03-16 00:29:31,47.843943,F,0,10.0,0.5,183.0,...,9.7,10.0,0.500000,183.00,10.600,NaN,201.00,3.30,141.0,9.70
2,6,107064,228232,2175-05-30 21:30:54,65.938398,F,0,65.0,10.0,181.0,...,10.6,65.0,10.000000,181.00,9.200,NaN,330.00,4.80,134.0,10.60
3,9,150750,220597,2149-11-09 13:07:02,41.787817,M,1,17.0,1.4,151.0,...,13.7,17.0,1.400000,151.00,12.900,2.333333,249.00,3.00,136.0,13.70
4,11,194540,229441,2178-04-16 06:19:32,50.146475,F,0,14.0,0.7,126.0,...,12.8,16.5,0.700000,127.50,12.550,NaN,250.50,3.85,141.5,11.05


In [5]:
import pandas as pd
import numpy as np

# 先做一個 key 表減少 merge 爆炸
keys = cohort[["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID", "INTIME"]].copy()

lab_itemids_flat = [50912,51006,51301,51222,51265,50983,50971,50931,50813]

labevents = pd.read_csv(
    "/content/drive/MyDrive/mimic-iii-clinical-database-1.4/LABEVENTS.csv.gz",
    usecols=["SUBJECT_ID", "HADM_ID", "ITEMID", "CHARTTIME", "VALUENUM"],
    parse_dates=["CHARTTIME"],
)

labevents = labevents[
    labevents["ITEMID"].isin(lab_itemids_flat)
].dropna(subset=["VALUENUM"])

# 只留 cohort 內的住院（大幅縮小）
labevents = labevents.merge(
    keys[["SUBJECT_ID","HADM_ID"]].drop_duplicates(),
    on=["SUBJECT_ID","HADM_ID"],
    how="inner"
)

# 再 join INTIME（這時候資料量已經小很多）
lab_df = labevents.merge(keys, on=["SUBJECT_ID","HADM_ID"], how="inner")

lab_df["hours_from_icu"] = (lab_df["CHARTTIME"] - lab_df["INTIME"]).dt.total_seconds()/3600
lab_df = lab_df[(lab_df["hours_from_icu"] >= 0) & (lab_df["hours_from_icu"] <= 24)]
print("lab_df after 24h filter:", lab_df.shape)


lab_df after 24h filter: (592263, 8)


In [6]:
# =====================
# Aggregate lab features
# =====================
lab_itemids = {
    "creatinine": [50912],
    "bun": [51006],
    "wbc": [51301],
    "hemoglobin": [51222],
    "platelet": [51265],
    "sodium": [50983],
    "potassium": [50971],
    "glucose": [50931],
    "lactate": [50813],
}

itemid_to_lab = {
    itemid: lab
    for lab, ids in lab_itemids.items()
    for itemid in ids
}

lab_df["lab"] = lab_df["ITEMID"].map(itemid_to_lab)

lab_feat = (
    lab_df
    .groupby(["ICUSTAY_ID", "lab"])["VALUENUM"]
    .agg(["min", "max", "mean"])
    .reset_index()
)

lab_feat.columns = ["ICUSTAY_ID", "lab", "lab_min", "lab_max", "lab_mean"]

# pivot wide
lab_feat_wide = lab_feat.pivot(
    index="ICUSTAY_ID",
    columns="lab",
    values=["lab_min", "lab_max", "lab_mean"]
)

lab_feat_wide.columns = [
    f"{stat}_{lab}" for stat, lab in lab_feat_wide.columns
]

lab_feat_wide = lab_feat_wide.reset_index()

print("lab_feat_wide:", lab_feat_wide.shape)
lab_feat_wide.head()


lab_feat_wide: (37420, 28)


,ICUSTAY_ID,lab_min_bun,lab_min_creatinine,lab_min_glucose,lab_min_hemoglobin,lab_min_lactate,lab_min_platelet,lab_min_potassium,lab_min_sodium,lab_min_wbc,...,lab_max_wbc,lab_mean_bun,lab_mean_creatinine,lab_mean_glucose,lab_mean_hemoglobin,lab_mean_lactate,lab_mean_platelet,lab_mean_potassium,lab_mean_sodium,lab_mean_wbc
0,200003,19.0,0.7,90.0,10.8,2.6,109.0,3.1,141.0,40.2,...,43.9,19.5,0.85,124.5,10.800000,3.933333,114.000000,3.15,142.5,42.050000
1,200007,8.0,0.8,217.0,13.3,NaN,247.0,3.8,135.0,11.2,...,11.2,9.0,0.80,225.0,13.300000,NaN,247.000000,3.85,136.5,11.200000
2,200009,15.0,0.5,91.0,9.2,0.7,81.0,3.9,143.0,10.5,...,14.3,15.5,0.50,91.0,10.233333,2.000000,150.166667,4.00,143.0,12.416667
3,200012,NaN,NaN,NaN,10.4,NaN,129.0,NaN,NaN,4.9,...,4.9,NaN,NaN,NaN,10.400000,NaN,129.000000,NaN,NaN,4.900000
4,200014,24.0,0.6,101.0,10.3,0.9,120.0,4.0,142.0,14.3,...,14.7,24.0,0.60,107.5,11.000000,0.900000,125.000000,4.10,143.0,14.500000


In [7]:
# =====================
# Merge features + impute
# =====================
model_df = cohort.merge(lab_feat_wide, on="ICUSTAY_ID", how="left")

# baseline features
model_df["GENDER"] = model_df["GENDER"].map({"M": 1, "F": 0})

feature_cols = [
    "age", "GENDER"
] + [c for c in model_df.columns if c.startswith(("lab_min", "lab_max", "lab_mean"))]

X = model_df[feature_cols]
y = model_df["death_30d"]

# median imputation（ICU 常用、穩）
X = X.fillna(X.median())

print("Final feature matrix:", X.shape)


Final feature matrix: (38509, 29)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=1000))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred)
pr_auc = average_precision_score(y_test, y_pred)

print(f"AUC: {auc:.3f}")
print(f"PR-AUC: {pr_auc:.3f}")


AUC: 0.776
PR-AUC: 0.381


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

# gender numeric
model_df = model_df.copy()
model_df["GENDER"] = model_df["GENDER"].map({"M": 1, "F": 0})

feature_cols = ["age", "GENDER"] + [c for c in model_df.columns if c.startswith(("lab_min", "lab_max", "lab_mean"))]
X = model_df[feature_cols].copy()
y = model_df["death_30d"].astype(int).copy()

# median impute (simple + medical-friendly)
X = X.fillna(X.median(numeric_only=True))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [11]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

# -------------------------
# 0) Build X, y (robust)
# -------------------------
model_df = model_df.copy()
model_df["GENDER"] = model_df["GENDER"].map({"M": 1, "F": 0}).astype("float64")  # keep numeric

feature_cols = ["age", "GENDER"] + [c for c in model_df.columns if c.startswith(("lab_min", "lab_max", "lab_mean"))]
X = model_df[feature_cols].copy()
y = model_df["death_30d"].astype(int).copy()

# 只保留 numeric 欄位（避免混到 object）
X = X.apply(pd.to_numeric, errors="coerce")

# 刪掉「整欄都是 NaN」的特徵（不然 median 也會 NaN）
all_nan_cols = X.columns[X.isna().all()].tolist()
if len(all_nan_cols) > 0:
    print("Dropping all-NaN columns:", all_nan_cols[:10], "..." if len(all_nan_cols) > 10 else "")
    X = X.drop(columns=all_nan_cols)

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -------------------------
# 1) Models (with imputer in pipeline)
# -------------------------
models = {
    "LogReg": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=3000, class_weight="balanced"))
    ]),
    "RandomForest": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("clf", RandomForestClassifier(
            n_estimators=500,
            min_samples_leaf=20,
            n_jobs=-1,
            class_weight="balanced_subsample",
            random_state=42
        ))
    ]),
    # 這個模型可吃 NaN，但我們還是 impute，確保一致性 & 好解釋
    "HistGBDT": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("clf", HistGradientBoostingClassifier(
            max_depth=6,
            learning_rate=0.05,
            max_iter=400,
            random_state=42
        ))
    ]),
}

def eval_model(name, model):
    model.fit(X_train, y_train)
    p = model.predict_proba(X_test)[:, 1]  # 這三個都有 predict_proba
    auc = roc_auc_score(y_test, p)
    pr_auc = average_precision_score(y_test, p)

    y_hat = (p >= 0.5).astype(int)
    return {
        "Model": name,
        "ROC_AUC": auc,
        "PR_AUC": pr_auc,
        "F1@0.5": f1_score(y_test, y_hat),
        "Precision@0.5": precision_score(y_test, y_hat, zero_division=0),
        "Recall@0.5": recall_score(y_test, y_hat, zero_division=0),
    }

res_df = pd.DataFrame([eval_model(n, m) for n, m in models.items()]) \
    .sort_values(["ROC_AUC", "PR_AUC"], ascending=False)

res_df


Dropping all-NaN columns: ['GENDER'] 


,Model,ROC_AUC,PR_AUC,F1@0.5,Precision@0.5,Recall@0.5
1,RandomForest,0.837451,0.468683,0.452747,0.358635,0.613826
2,HistGBDT,0.836178,0.469269,0.321881,0.666667,0.212157
0,LogReg,0.779579,0.377849,0.357854,0.244943,0.663886


In [12]:
res_df["Note"] = [
    "Best overall balance (chosen model)",
    "High precision, conservative detection",
    "Baseline, interpretable"
]
res_df


,Model,ROC_AUC,PR_AUC,F1@0.5,Precision@0.5,Recall@0.5,Note
1,RandomForest,0.837451,0.468683,0.452747,0.358635,0.613826,Best overall balance (chosen model)
2,HistGBDT,0.836178,0.469269,0.321881,0.666667,0.212157,"High precision, conservative detection"
0,LogReg,0.779579,0.377849,0.357854,0.244943,0.663886,"Baseline, interpretable"


In [13]:
best_model = models["RandomForest"]
best_model.fit(X_train, y_train)

model_df["pred_risk"] = best_model.predict_proba(X)[:, 1]

model_df.to_csv("icu_mortality_powerbi.csv", index=False)


baseline

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

# -------- Baseline features --------
Xb = model_df[["age", "GENDER"]].copy()
Xb = Xb.apply(pd.to_numeric, errors="coerce")

yb = model_df["death_30d"].astype(int)

Xb_train, Xb_test, yb_train, yb_test = train_test_split(
    Xb, yb, test_size=0.2, random_state=42, stratify=yb
)

baseline_lr = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced"))
])

baseline_lr.fit(Xb_train, yb_train)

pb = baseline_lr.predict_proba(Xb_test)[:, 1]

baseline_auc = roc_auc_score(yb_test, pb)
baseline_pr  = average_precision_score(yb_test, pb)

print("Baseline ROC-AUC:", baseline_auc)
print("Baseline PR-AUC :", baseline_pr)


Baseline ROC-AUC: 0.6342206233804215
Baseline PR-AUC : 0.16890305786836496


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['GENDER']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['GENDER']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Full

In [15]:
# -------- Full model (best RF) --------
best_rf = models["RandomForest"]
best_rf.fit(X_train, y_train)

pf = best_rf.predict_proba(X_test)[:, 1]

full_auc = roc_auc_score(y_test, pf)
full_pr  = average_precision_score(y_test, pf)

print("Full ROC-AUC:", full_auc)
print("Full PR-AUC :", full_pr)


Full ROC-AUC: 0.8374509838996037
Full PR-AUC : 0.46868339666081327


In [16]:
compare_df = pd.DataFrame([
    {
        "Model": "Baseline (Age + Gender)",
        "ROC_AUC": baseline_auc,
        "PR_AUC": baseline_pr,
        "Comment": "Demographic-only baseline"
    },
    {
        "Model": "Full (Age + Gender + 24h Labs, RandomForest)",
        "ROC_AUC": full_auc,
        "PR_AUC": full_pr,
        "Comment": "Clinical features significantly improve performance"
    }
])

compare_df


,Model,ROC_AUC,PR_AUC,Comment
0,Baseline (Age + Gender),0.634221,0.168903,Demographic-only baseline
1,"Full (Age + Gender + 24h Labs, RandomForest)",0.837451,0.468683,Clinical features significantly improve perfor...


In [17]:
model_df.to_csv("icu_full_model_powerbi.csv", index=False)


In [18]:
model_df["baseline_risk"] = baseline_lr.predict_proba(Xb)[:, 1]
model_df.to_csv("icu_baseline_full_powerbi.csv", index=False)


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['GENDER']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [19]:
final_df = model_df.copy()

# baseline prediction
final_df["baseline_risk"] = baseline_lr.predict_proba(Xb)[:, 1]

# full model prediction
final_df["full_risk"] = best_rf.predict_proba(X)[:, 1]

# 只留 Power BI 需要的欄位（乾淨、好用）
final_df = final_df[
    [
        "ICUSTAY_ID",
        "age",
        "GENDER",
        "death_30d",
        "baseline_risk",
        "full_risk",
    ]
]

final_df.to_csv("icu_powerbi_final.csv", index=False)


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['GENDER']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
